# WHIR: Reed–Solomon Proximity Testing with Super-Fast Verification

[WHIR](https://eprint.iacr.org/2024/1586) is a multivariate Polynomial Commitment Scheme (PCS) with three main ideas combined into one:

1. Using Sumcheck as a _proof_ of multilinear polynomial commitment. 
    Given, a multilinear polynomial $f(x_1, \cdots, x_\mu)$, it's 
    evaluation at a point $\vec{a}$ can be written as 
    $f(\vec{a}) = \sum_{\vec{b}\in \{0,1\}^\mu} f(\vec{b})\cdot \mathsf{eq}(\vec{b},\vec{a})$.
    The Sumcheck protocol _reduces_ a claim of this form to a 
    claim about evaluating $f(\vec{x})$ at a random point 
    $\vec{r}$ of verifier's choosing. But this causes a problem: 
    If the verifier could not evaluate $f(\vec{a})$ in the first place, 
    why would it be able to evaluate $f(\vec{r})$? 

    The first main idea in WHIR (which first appeared in 
    [BaseFold](https://eprint.iacr.org/2023/1705.pdf)) 
    is that instead of reducing the sumcheck claim to a _random evaluation_, 
    one can **reduce the sumcheck claim to a Proof of Proximity** and a final evaluation. 
    To achieve this, WHIR intermixes sumcheck rounds with FRI rounds, and relies on 
    consistency checks across FRI proof oracles to ensure soundness 
    across rounds. By the last round, the FRI proof oracle is small enough that
    the sumcheck value can be directly computed and checked against expected value. 
    This is the gist of BaseFold/WHIR reduction.

3. The second idea in WHIR is to use lower rate codes to minimize FRI query 
   complexity (this idea first appeared in [STIR](https://eprint.iacr.org/2024/390.pdf) paper).
   Unlike in STIR, however, WHIR elegantly avoids division operations during inter-round 
   consistency checks by simply updating the sumcheck claim on prover and verifier side. 

4. The third idea is to use out of domain (OOD) queries to eliminate "pretenders." This allows 
   WHIR to achieve better soundness even in in list-decoding region. This idea first 
   appeared in [DEEP-FRI](https://www.math.toronto.edu/swastik/deep-fri.pdf) paper.

The following code describes a run of the protocol. All the SageMath WHIR code is located in [src/whir.py](./src/whir.py).

File [src/proth_primes.py](./src/proth_primes.py) has a long list of [proth prime](https://en.wikipedia.org/wiki/Proth_prime), which are prime numbers of the form $k\cdot2^n + 1$ for different values of $k$ and $n$. These primes are convenient for NTT operations.

In [13]:
%load_ext autoreload
%autoreload 2

from src.proth_primes import proth_in_range
from src.whir import *
from sage.rings.finite_rings.all import GF
from sage.rings.polynomial.all import PolynomialRing
from sage.misc.functional import log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
(p, k, n) = proth_in_range(15, 15, 27,27)
print(f"Using prime {p} = {k}⋅2^{n} + 1")
Fq = GF(p)
omega = Fq.multiplicative_generator() 
omega = omega**k
omega_2sylow_expo = log(omega.multiplicative_order(), 2)
print(f"Maximal 2-sylow multiplicative generator: {omega}")
print(f"Maximal 2-sylow multiplicative power: {omega_2sylow_expo}")

Using prime 2013265921 = 15⋅2^27 + 1
Maximal 2-sylow multiplicative generator: 440564289
Maximal 2-sylow multiplicative power: 27


---
### Setup

Creating WHIR sumcheck with three variables and using a random multilinear polynomial.

In [15]:
variables_count = 3
code_rate_factor = 8
sumcheck_rounds = 2
shift_queries = 5

PolyRing = gen_mvariate_ring(Fq, variables_count=variables_count)
WeightRing = PolynomialRing(PolyRing, "Z") 

X0, X1, X2 = PolyRing.gens()
Z = WeightRing.gen()

input_poly = X0*X1*X2 - 2*X0*X1 + 3*X0*X2 - 4*X1*X2 + 5*X0 - 6*X2 + 7 # random_multilinear_poly(PolyRing);
evaluation_point = [5, 6, 7] # [Fq.random_element() for _ in range(variables_count)]

pcs_evaluation_claim = input_poly(evaluation_point)

print(f"Using polynomial: {input_poly}")
print(f"Evaluation point: {evaluation_point}")
print(f"Evaluated value: {input_poly(evaluation_point)}")

Using polynomial: X₀*X₁*X₂ - 2*X₀*X₁ + 3*X₀*X₂ - 4*X₁*X₂ + 5*X₀ - 6*X₂ + 7
Evaluation point: [5, 6, 7]
Evaluated value: 77


In [16]:
ntt_order = code_rate_factor*(2**variables_count)
omega_order = omega.multiplicative_order()
assert omega_order % ntt_order == 0
coset_exponent = omega_order // ntt_order
ntt_omega = omega**coset_exponent
print(f"Using NTT group generator: {ntt_omega}, order: {ntt_omega.multiplicative_order()}")

Using NTT group generator: 1721589904, order: 64


Create the weight polynomial for the given evaluation point. This evaluation point will typically come from the verifier, and is known to the verifier. 

In [17]:
weight_poly = eq_weight_polynomial(WeightRing, evaluation_point)
print(f"Prover's Weight polynomial: {weight_poly}")

Prover's Weight polynomial: (1287*X₀*X₁*X₂ - 594*X₀*X₁ - 585*X₀*X₂ - 572*X₁*X₂ + 270*X₀ + 264*X₁ + 260*X₂ - 120)*Z


### Verifier (positive case)

In [18]:
prover = WhirRound(input_poly, weight_poly, ntt_omega, pcs_evaluation_claim)
print(f"Sumcheck polynomial: {prover.sumcheck_poly}")

Sumcheck polynomial: 1287*X₀^2*X₁^2*X₂^2 - 3168*X₀^2*X₁^2*X₂ + 3276*X₀^2*X₁*X₂^2 - 5720*X₀*X₁^2*X₂^2 + 1188*X₀^2*X₁^2 + 6093*X₀^2*X₁*X₂ + 3784*X₀*X₁^2*X₂ - 1755*X₀^2*X₂^2 - 6838*X₀*X₁*X₂^2 + 2288*X₁^2*X₂^2 - 3510*X₀^2*X₁ - 528*X₀*X₁^2 - 2115*X₀^2*X₂ + 8785*X₀*X₁*X₂ - 1056*X₁^2*X₂ + 4290*X₀*X₂^2 + 2392*X₁*X₂^2 + 1350*X₀^2 - 2598*X₀*X₁ - 4775*X₀*X₂ - 5108*X₁*X₂ - 1560*X₂^2 + 1290*X₀ + 1848*X₁ + 2540*X₂ - 840


In [19]:
verifier = WhirVerifier(weight_poly, ntt_omega, pcs_evaluation_claim)
succes = verifier.validate_pcs_claim(prover)
if succes == True:
    print(f"PCS verification succeeded")
else:
    print(f"PCS verification failed")


----- New Sumcheck Round -----

Current Sumcheck claim: 77
Round-0 poly: 504*X₀^2 - 2051*X₀ + 812, challenge: None
Hypercube sum 77 matches claimed sum 77
Round-1 poly: 710417221*X₁^2 + 933868125*X₁ - 122024773, challenge: 913189950
Hypercube sum -613030121 matches claimed sum -613030121
Updated Sumcheck claim: -613030121

----- Next OOD + STIR Round -----

Current Sumcheck claim: -613030121
OOD challenge: 1961264585
Shift queries: [1835875777, 1253260071]
Combination randomness 𝛾: 1576224895

    P> Weight update for OOD challenge 1961264585 => 424934549 : Z*(-70261355*X₁*X₂ - 16870659*X₁ + 726465213*X₂ - 337231938)
    P> Weight update for shift query 1835875777 : Z*(633836656*X₁*X₂ + 512324488*X₁ + 76728129*X₂ + 50331008)
    P> Weight update for shift query 1253260071 : Z*(191116099*X₁*X₂ - 855563900*X₁ + 100838210*X₂ - 677049140)
    P> Updated new weight: (417065636*X₁*X₂ - 463474140*X₁ + 745632449*X₂ - 226734683)*Z
    P> Sumcheck claim combination randomness additional sum : 1

### Verifier (negative case)

In [20]:
prover = WhirRound(input_poly, weight_poly, ntt_omega, pcs_evaluation_claim)

verifier = WhirVerifier(weight_poly, ntt_omega, pcs_evaluation_claim + 1)
succes = verifier.validate_pcs_claim(prover)
if succes == True:
    print(f"PCS verification succeeded")
else:
    print(f"PCS verification failed")


----- New Sumcheck Round -----

Current Sumcheck claim: 78
Round-0 poly: 504*X₀^2 - 2051*X₀ + 812, challenge: None
ERROR: Sumcheck claim failed
PCS verification failed
